In [40]:
import requests
import re
import numpy as np
import pandas as pd
import os
import json

In [36]:
Area = 'CountyStatistics'
Stat_type = 'GetDroughtSeverityStatisticsByAreaPercent'
BASE_URL = f'https://usdmdataservices.unl.edu/api/{Area}/{Stat_type}'

In [37]:
# Other parameters
params = {
    'aoi' : 'total', #Area of interest, FIPS code for NY
    'dx' : '1',
    'DxLevelThresholdFrom' : '10',
    'DxLevelThresholdTo' : '70',
    'startdate' : '1/1/2021', # M/D/YYYY
    'enddate' : '2/1/2021',
    'statisticsType' : '1' # 1 for traditional or 2 for categorical. 
}


In [38]:
req = requests.get(BASE_URL, params=params)

In [39]:
req.json()

[]

In [ ]:
# Saving the file
save_path = '../../../../../../../Google Drive/My Drive/Capstone private/USDM/'
os.makedirs(save_path, exist_ok=True)

In [ ]:
with open(save_path, 'w') as f:
    json.dump(req.json(),f)